In [8]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
import requests
!pip install geocoder

import geocoder

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from bs4 import BeautifulSoup

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from urllib.request import urlopen
import requests
import ssl

print('Libraries imported.')

     |████████████████████████████████| 102kB 6.8MB/s ta 0:00:011
Solving environment: \ ^C
failed

CondaError: KeyboardInterrupt

Libraries imported.


<h2> Extract and Clean the Data <h2>

In [9]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
extracting_data = requests.get(url).text
wiki_data = BeautifulSoup(extracting_data, 'lxml')

In [10]:
column_names = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)

content = wiki_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

In [11]:
toronto = toronto[toronto.Borough!='Not assigned\n']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1

In [12]:
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.replace(r'\n',  ' ', regex=True, inplace=True)
df

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [13]:
def get_latilong(postal_code):
    lati_long_coords = None
    while(lati_long_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lati_long_coords = g.latlng
    return lati_long_coords
    
get_latilong('M4G')

[43.70941386000004, -79.36309957799995]

In [14]:
postal_codes = df['Postalcode']    
coords = [ get_latilong(postal_code) for postal_code in postal_codes.tolist() ]
# Adding Columns Latitude & Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [15]:
df.head(10)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.808626,-79.189913
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.785779,-79.157368
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765806,-79.185284
3,M1G,Scarborough,Woburn,43.771545,-79.218135
4,M1H,Scarborough,Cedarbrae,43.768791,-79.238813
5,M1J,Scarborough,Scarborough Village,43.744203,-79.228725
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.726881,-79.265694
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.713340,-79.284942
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.723538,-79.228353
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.696448,-79.265642


<h2> Map of Toronto and Downtown Toronto <h2>

In [63]:
address = 'Toronto,Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude_x = location.latitude
longitude_y = location.longitude
print('The Geograpical Co-ordinate of Downtown Toronto, Toronto are {}, {}.'.format(latitude_x, longitude_y))

The Geograpical Co-ordinate of Downtown Toronto, Toronto are 43.6534817, -79.3839347.


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


In [65]:
map_Toronto = folium.Map(location=[latitude_x, longitude_y], zoom_start=10)
for lat, lng, nei in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    
    label = '{}'.format(nei)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [26]:
downtowntoronto_data = df[df['Borough'].str.contains("Downtown Toronto")].reset_index(drop=True)
print(toronto_data.shape)
downtowntoronto_data.head()

(19, 5)


,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.681893,-79.376706
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667656,-79.367326
2,M4Y,Downtown Toronto,Church and Wellesley,43.666659,-79.381472
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529


In [27]:
map_downtowntoronto = folium.Map(location=[latitude_x, longitude_y], zoom_start=11)

for lat, lng, label in zip(downtowntoronto_data['Latitude'], downtowntoronto_data['Longitude'], downtowntoronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng], radius=5, popup=label, color='blue', fill=True, fill_color='#3186cc', fill_opacity=0.7,parse_html=False).add_to(map_downtowntoronto)  
map_downtowntoronto

<h2> Venues in Neighborhoods of Downtown Toronto <h2>

In [28]:
CLIENT_ID = 'WWEA4QKSWTD0OB3134RLDFP0QL3WS2L0HCIN2JJIPHX3A4W2' 
CLIENT_SECRET = 'QMWUVYA21UTA15U0TKLSSPJBUGWS3Q22IIP0JEDUZOR1DKSV' 
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WWEA4QKSWTD0OB3134RLDFP0QL3WS2L0HCIN2JJIPHX3A4W2
CLIENT_SECRET:QMWUVYA21UTA15U0TKLSSPJBUGWS3Q22IIP0JEDUZOR1DKSV


In [29]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([( name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'], v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)

In [34]:
df1 = downtowntoronto_data
downtowntoronto_venues = getNearbyVenues(names=df1['Neighborhood'], latitudes=df1['Latitude'],longitudes=df1['Longitude'])

Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Queen's Park, Ontario Provincial Government


In [35]:
downtowntoronto_venues.shape

(496, 7)

In [36]:
downtowntoronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.681893,-79.376706,Summerhill Market,43.686265,-79.375458,Grocery Store
1,Rosedale,43.681893,-79.376706,Rosedale Park,43.682328,-79.378934,Playground
2,Rosedale,43.681893,-79.376706,Whitney Park,43.682036,-79.373788,Park
3,Rosedale,43.681893,-79.376706,Scoops Convenience Boutique,43.686148,-79.375828,Candy Store
4,"St. James Town, Cabbagetown",43.667656,-79.367326,Cranberries,43.667843,-79.369407,Diner


In [38]:
downtowntoronto_venues.groupby('Neighborhood', as_index=False).count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,30,30,30,30,30,30
1,"CN Tower, King and Spadina, Railway Lands, Har...",30,30,30,30,30,30
2,Central Bay Street,30,30,30,30,30,30
3,Christie,12,12,12,12,12,12
4,Church and Wellesley,30,30,30,30,30,30
5,"Commerce Court, Victoria Hotel",30,30,30,30,30,30
6,"First Canadian Place, Underground city",30,30,30,30,30,30
7,"Garden District, Ryerson",30,30,30,30,30,30
8,"Harbourfront East, Union Station, Toronto Islands",5,5,5,5,5,5
9,"Kensington Market, Chinatown, Grange Park",30,30,30,30,30,30


In [41]:
print('There are {} unique categories.'.format(len(downtowntoronto_venues['Venue Category'].unique())))

There are 136 unique categories.


<h2> One Hot Encoding of Features <h2>

In [42]:
downtowntoronto_onehot = pd.get_dummies(downtowntoronto_venues[['Venue Category']], prefix="", prefix_sep="")

downtowntoronto_onehot['Neighborhood'] = downtowntoronto_venues['Neighborhood'] 

fixed_columns = [downtowntoronto_onehot.columns[-1]] + list(downtowntoronto_onehot.columns[:-1])
downtowntoronto_onehot = downtowntoronto_onehot[fixed_columns]

downtowntoronto_onehot.head()

,Yoga Studio,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Belgian Restaurant,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Distribution Center,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,Hobby Shop,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Monument / Landmark,Museum,Music Venue,Neighborhood,New American Restaurant,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pub,Ramen Restaurant,Record Shop,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Smoke Shop,Spa,Speakeasy,Steakhouse,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"St. James Town, Cabbagetown",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [43]:
downtowntoronto_grouped = downtowntoronto_onehot.groupby('Neighborhood').mean().reset_index()
downtowntoronto_grouped

,Neighborhood,Yoga Studio,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Belgian Restaurant,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Distribution Center,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,Hobby Shop,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Monument / Landmark,Museum,Music Venue,New American Restaurant,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pub,Ramen Restaurant,Record Shop,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Smoke Shop,Spa,Speakeasy,Steakhouse,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.000000,0.00,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333,0.066667,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.00,0.000000,0.066667,0.033333,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.033333,0.033333,0.033333,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.00,0.033333,0.000000,0.0,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.033333,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.00,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.0,0.033333,0.000000,0.033333,0.033333,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000

In [44]:
num_top_venues = 5
for neigh in downtowntoronto_grouped['Neighborhood']:
    print("----"+neigh+"----")
    temp = downtowntoronto_grouped[downtowntoronto_grouped['Neighborhood'] == neigh].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Berczy Park----
                venue  freq
0        Cocktail Bar  0.07
1  Seafood Restaurant  0.07
2            Beer Bar  0.07
3              Bistro  0.03
4         Cheese Shop  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                  venue  freq
0                  Park  0.07
1  Gym / Fitness Center  0.07
2                  Café  0.07
3    Italian Restaurant  0.07
4         Historic Site  0.03


----Central Bay Street----
         venue  freq
0  Coffee Shop  0.13
1         Café  0.07
2        Plaza  0.07
3        Hotel  0.03
4   Restaurant  0.03


----Christie----
           venue  freq
0  Grocery Store  0.25
1           Café  0.25
2           Park  0.08
3     Playground  0.08
4    Coffee Shop  0.08


----Church and Wellesley----
                venue  freq
0         Coffee Shop  0.10
1         Men's Store  0.07
2   Martial Arts Dojo  0.03
3  Salon / Barbershop  0.03
4     Bubble Tea Shop  0.03


--

<h2> Most Common Venues in each Neighborhood <h2>

In [45]:
def  return_most_common_venues(row, num_top_venues):
 
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [50]:
import numpy as np
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] =downtowntoronto_grouped['Neighborhood']

for ind in np.arange(downtowntoronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtowntoronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Cocktail Bar,Seafood Restaurant,Beer Bar,Farmers Market,Café,Park,Breakfast Spot,Cheese Shop,Restaurant,Comfort Food Restaurant
1,"CN Tower, King and Spadina, Railway Lands, Har...",Gym / Fitness Center,Café,Italian Restaurant,Park,New American Restaurant,Mexican Restaurant,Restaurant,Ramen Restaurant,Brewery,Peruvian Restaurant
2,Central Bay Street,Coffee Shop,Café,Plaza,Spa,Clothing Store,Middle Eastern Restaurant,Modern European Restaurant,Comic Shop,Japanese Restaurant,Bubble Tea Shop
3,Christie,Grocery Store,Café,Park,Athletics & Sports,Playground,Candy Store,Baby Store,Coffee Shop,Garden,Diner
4,Church and Wellesley,Coffee Shop,Men's Store,Bookstore,Bubble Tea Shop,Restaurant,Ramen Restaurant,Pub,Breakfast Spot,Pizza Place,Dance Studio
5,"Commerce Court, Victoria Hotel",Café,Japanese Restaurant,Coffee Shop,Hotel,Gastropub,Tailor Shop,Deli / Bodega,Pub,Ice Cream Shop,Restaurant
6,"First Canadian Place, Underground city",Café,Restaurant,Coffee Shop,Seafood Restaurant,Bookstore,Gym / Fitness Center,Pub,Ice Cream Shop,Sandwich Place,Deli / Bodega
7,"Garden District, Ryerson",Café,Ramen Restaurant,Theater,Hotel,Diner,Plaza,Japanese Restaurant,Burger Joint,Burrito Place,Comic Shop
8,"Harbourfront East, Union Station, Toronto Islands",Theme Park,Park,Harbor / Marina,Farm,Creperie,Distribution Center,Diner,Dessert Shop,Deli / Bodega,Dance Studio
9,"Kensington Market, Chinatown, Grange Park",Café,Mexican Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Coffee Shop,Pizza Place,Caribbean Restaurant,Cheese Shop,Cocktail Bar,Comfort Food Restaurant


<h2> K Means Clustering <h2>

In [51]:
from sklearn.cluster import KMeans
import sklearn.cluster.k_means_
km = KMeans(n_clusters=3, init='k-means++', max_iter=100, n_init=1, 
  verbose=True)

In [55]:
kclusters = 10
downtowntoronto_grouped_clustering = downtowntoronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(downtowntoronto_grouped_clustering)
print(kmeans.labels_[0:10])
print(len(kmeans.labels_))


[6 0 9 5 0 3 3 9 1 6]
19


In [59]:
neighborhoods_venues_sorted.insert(0, 'Clusters Labels', kmeans.labels_)
downtowntoronto_merged = df1
downtowntoronto_merged = downtowntoronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtowntoronto_merged.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Clusters Labels,Cluster Labelss,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.681893,-79.376706,2,2,2,Park,Grocery Store,Playground,Candy Store,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,College Gym,Farm
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667656,-79.367326,0,0,0,Restaurant,Coffee Shop,Italian Restaurant,Bakery,Café,Jewelry Store,Japanese Restaurant,Pub,Deli / Bodega,Playground
2,M4Y,Downtown Toronto,Church and Wellesley,43.666659,-79.381472,0,0,0,Coffee Shop,Men's Store,Bookstore,Bubble Tea Shop,Restaurant,Ramen Restaurant,Pub,Breakfast Spot,Pizza Place,Dance Studio
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041,7,7,7,Pub,Café,Athletics & Sports,Mexican Restaurant,Mediterranean Restaurant,Distribution Center,Performing Arts Venue,Seafood Restaurant,Hotel,Chocolate Shop
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529,9,9,9,Café,Ramen Restaurant,Theater,Hotel,Diner,Plaza,Japanese Restaurant,Burger Joint,Burrito Place,Comic Shop


In [60]:
downtowntoronto_merged.drop(columns = ['Cluster Labelss', 'Clusters Labels'])

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.681893,-79.376706,2,Park,Grocery Store,Playground,Candy Store,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,College Gym,Farm
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667656,-79.367326,0,Restaurant,Coffee Shop,Italian Restaurant,Bakery,Café,Jewelry Store,Japanese Restaurant,Pub,Deli / Bodega,Playground
2,M4Y,Downtown Toronto,Church and Wellesley,43.666659,-79.381472,0,Coffee Shop,Men's Store,Bookstore,Bubble Tea Shop,Restaurant,Ramen Restaurant,Pub,Breakfast Spot,Pizza Place,Dance Studio
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041,7,Pub,Café,Athletics & Sports,Mexican Restaurant,Mediterranean Restaurant,Distribution Center,Performing Arts Venue,Seafood Restaurant,Hotel,Chocolate Shop
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529,9,Café,Ramen Restaurant,Theater,Hotel,Diner,Plaza,Japanese Restaurant,Burger Joint,Burrito Place,Comic Shop
5,M5C,Downtown Toronto,St. James Town,43.651734,-79.375554,3,Gastropub,Café,Coffee Shop,Ice Cream Shop,Farmers Market,Cocktail Bar,New American Restaurant,Optical Shop,Park,Cosmetics Shop
6,M5E,Downtown Toronto,Berczy Park,43.645196,-79.373855,6,Cocktail Bar,Seafood Restaurant,Beer Bar,Farmers Market,Café,Park,Breakfast Spot,Cheese Shop,Restaurant,Comfort Food Restaurant
7,M5G,Downtown Toronto,Central Bay Street,43.656072,-79.385653,9,Coffee Shop,Café,Plaza,Spa,Clothing Store,Middle Eastern Restaurant,Modern European Restaurant,Comic Shop,Japanese Restaurant,Bubble Tea Shop
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650542,-79.384116,4,Coffee Shop,Café,Seafood Restaurant,Speakeasy,Lounge,Monument / Landmark,Concert Hall,Opera House,Japanese Restaurant,Deli / Bodega
9,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.623750,-79.369200,1,Theme Park,Park,Harbor / Marina,Farm,Creperie,Distribution Center,Diner,Dessert Shop,Deli / Bodega,Dance Studio


<h2> Map of Clusters <h2>

In [62]:
map_clusters = folium.Map(location=[latitude_x, longitude_y], zoom_start=11)

xs = np.arange(kclusters)
ys = [i+xs+(i*xs)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(downtowntoronto_merged['Latitude'], downtowntoronto_merged['Longitude'], downtowntoronto_merged['Neighborhood'],kmeans.labels_):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon], radius=5, popup=label, color=rainbow[cluster-1], fill=True, fill_color=rainbow[cluster-1], fill_opacity=0.7).add_to(map_clusters)
map_clusters